In [15]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Boolean, DateTime
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime


In [16]:

# Connexion à MariaDB via SQLAlchemy
def create_connection_sqlalchemy(host_name, user_name, user_password, db_name=None, port=3306):
    try:
        db_url = f"mysql+pymysql://{user_name}:{user_password}@{host_name}:{port}/{db_name}"
        engine = create_engine(db_url, pool_recycle=3600)
        connection = engine.connect()
        print(f"Connexion réussie à la base de données {db_name}" if db_name else "Connexion réussie au serveur MariaDB")
        return connection, engine
    except SQLAlchemyError as e:
        print(f"Erreur : '{e}'")
        return None, None


In [17]:

def creer_table_si_absente(connection, engine, nom_table, colonnes, dtype):
    """
    Crée une table si elle n'existe pas déjà dans la base de données.
    """
    metadata = MetaData()
    colonnes_avec_types = []

    for colonne in colonnes:
        col_type = dtype.get(colonne, String(255))
        if col_type == 'int':
            col_type = Integer
        elif col_type == 'float':
            col_type = Float
        elif col_type == 'bool':
            col_type = Boolean
        elif col_type == 'datetime':
            col_type = DateTime
        else:
            col_type = String(255)
        colonnes_avec_types.append(Column(colonne, col_type))

    table = Table(nom_table, metadata, *colonnes_avec_types)
    metadata.create_all(engine)

In [31]:

def inserer_donnees(connection, nom_table, colonnes, donnees):
    """
    Insère les données dans la table MariaDB via SQLAlchemy.
    """
    metadata = MetaData()
    table = Table(nom_table, metadata, autoload_with=connection)
    
    # Convertir les données en une liste de dictionnaires
    data_dicts = [dict(zip(colonnes, row)) for row in donnees]
    
    try:
        connection.execute(table.insert(), data_dicts)
        connection.commit()
        print(f"{len(donnees)} lignes insérées avec succès dans la table {nom_table}")

    except SQLAlchemyError as e:
        print(f"Erreur lors de l'insertion : '{e}'")

In [19]:

def extraire_info_du_nom_fichier(fichier,liste_nom_table):  
    # extraction informations du nom de fichier à encoder dans la table
    annee_mois=fichier[:7]
    info_geo=fichier[8:-4]
    # where sans l'élément qui est dans la liste
    for i in liste_nom_table:
        if i in info_geo:
            info_geo=info_geo.replace(i,"")[:-1]
    return annee_mois,info_geo


In [20]:

def nettoyer_noms_colonnes(colonnes):
    """
    Nettoie les noms des colonnes pour éviter les problèmes de requêtes SQL.
    """
     # nettoyage des noms de colonnes
    colonnes = [colonne.replace(' ', '') for colonne in colonnes]
    colonnes = [colonne.replace('-', '') for colonne in colonnes]
    colonnes = [colonne.replace('_', '') for colonne in colonnes]
    colonnes_nettoyees = [colonne.replace(' ', '').replace('-', '').replace('_', '') for colonne in colonnes]
    return colonnes_nettoyees


In [21]:

def nettoyer_nom_table(liste_nom_table,chemin_fichier):
    for nom_table in liste_nom_table:
        if nom_table in chemin_fichier:
            nom_table = nom_table.replace('-', '')
            break
        else:
            nom_table = 'autre'
            
    print(f"nom de la table : {nom_table}")
    # Créer la table si elle n'existe pas
    return nom_table


In [22]:
def nettoyer_donnees(donnees):
    """
    Remplace les NaN dans les données par des valeurs par défaut.
    Pour les chaînes, remplace par une chaîne vide, pour les numériques, par NULL.
    """

    # extraire les doublons et supprimer les doublons - remplacer les nan
    duplicates=donnees[donnees.duplicated()]
    duplicates=duplicates.replace({np.nan:''})
    
    donnees=donnees.drop_duplicates()
    donnees = donnees.replace({np.nan: ''})
        
    return donnees, duplicates


In [23]:

def rajouter_un_index_table(nom_table,index_dict,df):
     # Ajouter une colonne d'index unique dans les tables
    if nom_table not in index_dict:
        index_dict[nom_table] = 1
    df['id'] = range(index_dict[nom_table], index_dict[nom_table] + len(df))
    index_dict[nom_table] += len(df)
    return df,index_dict


In [24]:

def traiter_fichier(connection, engine, chemin_fichier, fichier, liste_nom_table, dtype, index_dict, liste_col_a_supprimer):
    """
    Traite un fichier CSV en créant une table correspondante et en y insérant les données.
    """
    na_values = ['NA', 'N/A', '']
    df = pd.read_csv(chemin_fichier, dtype={col: typ for col, typ in dtype.items() if typ != 'datetime'}, na_values=na_values)

    # Supprimer les colonnes de la liste 'liste_col_a_supprimer'
    df = df.drop(columns=[col for col in liste_col_a_supprimer if col in df.columns])

    # Extraire et ajouter des informations au dataframe
    annee_mois, info_geo = extraire_info_du_nom_fichier(fichier, liste_nom_table)
    df['annee_mois'] = annee_mois
    df['info_geo'] = info_geo

    # Nettoyer les données
    df, duplicates = nettoyer_donnees(df)
    duplicates['source'] = fichier

    nom_table = nettoyer_nom_table(liste_nom_table, chemin_fichier) + "_temp"
    nom_table_duplicates = nom_table + "_duplicates"

    # Ajouter un index unique
    df, index_dict = rajouter_un_index_table(nom_table, index_dict, df)
    colonnes = nettoyer_noms_colonnes(list(df.columns))
    creer_table_si_absente(connection, engine, nom_table, colonnes, dtype)
    inserer_donnees(connection, nom_table, colonnes, df.values.tolist())

    # Traiter les doublons
    duplicates, index_dict = rajouter_un_index_table(nom_table_duplicates, index_dict, duplicates)
    colonnes_duplicates = nettoyer_noms_colonnes(list(duplicates.columns))
    creer_table_si_absente(connection, engine, nom_table_duplicates, colonnes_duplicates, dtype)
    inserer_donnees(connection, nom_table_duplicates, colonnes_duplicates, duplicates.values.tolist())

    print(f"Traitement du fichier : {chemin_fichier} terminé.")
    return df, index_dict


In [25]:

def parcourir_arborescence(connection,engine,chemin_racine, db_path,liste_nom_table,dtype,index_dict,liste_col_a_supprimer):
    """
    Parcourt récursivement l'arborescence et traite chaque fichier CSV trouvé.
    """
    for racine, sous_repertoires, fichiers in os.walk(chemin_racine):
        print(f"racine : {racine}")
        print(f"sous-repertoires : {sous_repertoires}")
        print(f"fichiers : {fichiers}")

        for fichier in fichiers:
            if fichier.endswith(".csv"):
                chemin_fichier = os.path.join(racine, fichier)
                print(f"chemin du fichier : {chemin_fichier}")
                df, index_dict=traiter_fichier(connection,engine,chemin_fichier, fichier,liste_nom_table,dtype, index_dict,liste_col_a_supprimer)
                
    return df, index_dict



In [32]:

chemin_racine = "C:/Users/Admin.local/Documents/projetint/files_test"

db_name = "crime_short" 

# on définit le nom des tables en fonction du nom du fichier (terminaison)
liste_nom_table = ['outcomes','stop-and-search','street']

# encodage des types en fonction de la colonne
dtype={'Longitude': 'float',
        'Latitude': 'float',
        'id':'int',
        'Partofapolicingoperation': 'bool',
        'Date': 'datetime',
        'Outcomelinkedtoobjectofsearch': 'bool',
        'Removalofmorethanjustouterclothing': 'bool'
}

liste_col_a_supprimer=['Falls within']

# Dictionnaire pour mémoriser l'index pour chaque table
index_dict = {}

# Connexion à la base de données via SQLAlchemy
connection, engine = create_connection_sqlalchemy("127.0.0.1", "root", "",db_name)

if connection:
    df, index_dict = parcourir_arborescence(connection,engine, chemin_racine, db_name, liste_nom_table, dtype, index_dict, liste_col_a_supprimer)
    
    # Fermer la connexion proprement
    connection.close()
    print("Connexion fermée.")
  
print(index_dict)


Connexion réussie à la base de données crime_short
racine : C:/Users/Admin.local/Documents/projetint/files_test
sous-repertoires : ['files']
fichiers : []
racine : C:/Users/Admin.local/Documents/projetint/files_test\files
sous-repertoires : []
fichiers : ['2019-11-bedfordshire-outcomes.csv', '2019-11-bedfordshire-stop-and-search.csv', '2019-11-bedfordshire-street.csv', '2021-10-avon-and-somerset-outcomes.csv', '2021-10-avon-and-somerset-stop-and-search.csv', '2021-10-avon-and-somerset-street.csv']
chemin du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2019-11-bedfordshire-outcomes.csv
nom de la table : outcomes
3050 lignes insérées avec succès dans la table outcomes_temp
25 lignes insérées avec succès dans la table outcomes_temp_duplicates
Traitement du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2019-11-bedfordshire-outcomes.csv terminé.
chemin du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2019-11-bedfordshire-sto

C:\Users\Admin.local\AppData\Local\Temp\ipykernel_18384\1416925358.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  duplicates=duplicates.replace({np.nan:''})


5258 lignes insérées avec succès dans la table street_temp
218 lignes insérées avec succès dans la table street_temp_duplicates
Traitement du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2019-11-bedfordshire-street.csv terminé.
chemin du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2021-10-avon-and-somerset-outcomes.csv
nom de la table : outcomes
6890 lignes insérées avec succès dans la table outcomes_temp
482 lignes insérées avec succès dans la table outcomes_temp_duplicates
Traitement du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2021-10-avon-and-somerset-outcomes.csv terminé.
chemin du fichier : C:/Users/Admin.local/Documents/projetint/files_test\files\2021-10-avon-and-somerset-stop-and-search.csv
nom de la table : stopandsearch
642 lignes insérées avec succès dans la table stopandsearch_temp
36 lignes insérées avec succès dans la table stopandsearch_temp_duplicates
Traitement du fichier : C:/Users/Admin.local/Do